<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week9_Day2_lesson2_tuto_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building AI Chat Agents with LangChain, Mistral & Tavily

## Overview

In this tutorial, you will build a tool-enabled conversational agent using LangChain and the Mistral LLM. By combining the ReAct reasoning framework with custom tools—web search and a calculator—you’ll create an agent that can:

* Dynamically call external services as needed.
* Reason through tasks step-by-step.
* Stream responses for a responsive user experience.

### Tutorial Steps

1. **Prerequisites**: Set up Python, a virtual environment, and account access.
2. **Install Required Packages**: Add LangChain, community extensions, LangGraph, and Mistral bindings.
3. **Configure Environment Variables**: Securely load API keys for LangSmith, Tavily, and Mistral.
4. **Build a Search Tool**: Wrap the Tavily Search API as a LangChain Tool.
5. **Define a Calculator Tool**: Create a safe arithmetic evaluator available to the agent.
6. **Initialize the Mistral Chat Model**: Connect to Mistral via LangChain’s helper.
7. **Build a ReAct Agent**: Enable the agent to interleave thought and action cycles.
8. **Invoke the Agent**: Send queries and observe the reasoning trace and final answer.
9. **High-Level API with LangGraph**: Use a simplified executor interface.
10. **Streamed Outputs**: Display partial responses in real time.

Each step explains both the code and the underlying purpose, guiding you from setup to a fully functional, interactive AI agent.


## 1. Prerequisites

Before you begin, ensure you have:

* **Python 3.8+** installed.
* A virtual environment (venv, conda, etc.) for isolating dependencies.
* API keys for:

  * **LangSmith** (tracing/debugging)
  * **Tavily Search**
  * **Mistral AI** (Mistral endpoint)
* (Optional) A Jupyter or Colab environment.

## 2. Install Required Packages

In [1]:
# Core LangChain + community extensions + Mistral
%pip install --quiet -U langchain langchain-community langchain-anthropic \
               langgraph langgraph-checkpoint-sqlite tavily-python \
               "langchain[mistralai]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.5 MB/s eta 0:00:00


* **`pip install`**: The Python package installer.
* **`--quiet`**: Reduces the amount of output shown (useful for cleaner logs).
* **`-U`** (or `--upgrade`): Ensures you get the latest version of each package.

The backslashes (`\`) let you split the command over multiple lines for readability. The packages are:

- 1. **`langchain`** : The core LangChain framework for building LLM applications (chains, agents, prompt templates, etc.).
- 2. **`langchain-community`** : Community-contributed extensions and integrations (additional tools, helpers, etc.).
- 3. **`langchain-anthropic`** : Support for Anthropic models if you plan to use them alongside Mistral.
- 4. **`langgraph`** : A higher-level orchestration layer built on top of LangChain, useful for more complex pipelines.
- 5. **`langgraph-checkpoint-sqlite`** : A plugin for `langgraph` that enables checkpointing state in a SQLite database, so you can resume or debug runs.
- 6. **`tavily-python`** : The Python SDK for the Tavily Search API, which we will use to add web-search capabilities to our agent.
- 7. **`"langchain[mistralai]"`** : An “extra” in pip syntax. This installs additional dependencies needed to connect to Mistral endpoints (e.g., HTTP client, authentication helpers).

After running this command, you’ll have all the libraries installed and ready to import in your Python scripts or notebooks.

## 3. Configure Environment Variables

Before your code can call external services, you need to provide your API keys securely. We do this using environment variables and the `getpass` module to prompt without echoing your keys on screen.


### What this does:

* **`os.environ[...] = ...`** sets an environment variable in your Python session.
* **`getpass.getpass(...)`** securely prompts you to type the key without showing it in the console.
* **`LANGSMITH_TRACING`** toggles verbose tracing in LangSmith’s dashboard.
* Checking `if not os.environ.get(...)` lets you pre‑set keys in your shell (`export KEY=...`) or a `.env` file if you prefer.

### Obtaining API Keys
To use these services, sign up and generate keys on their respective websites:
* **LangSmith**:
  1. Visit **[https://smith.langchain.com](https://smith.langchain.com)**
  2. Sign in or create an account.
  3. Navigate to **Account > API Keys**.
  4. Click **New Key**, give it a name, and copy the generated value.
  5. In your Python session or shell, set:

     ```bash
     export LANGSMITH_API_KEY="<your-langsmith-key>"
     export LANGSMITH_TRACING="true"
     ```

* **Tavily Search**:
    1. Go to **[https://developer.tavily.com](https://developer.tavily.com)** (or the Tavily website).
    2. Sign up for a developer account.
    3. In the dashboard, find **API Keys** and click **Create Key**.
    4. Copy the key and set:

     ```bash
     export TAVILY_API_KEY="<your-tavily-key>"
     ```

* **Mistral AI**:
  1. Navigate to **[https://platform.mistral.ai](https://platform.mistral.ai)**.
  2. Register or log in to your account.
  3. Visit **API Keys** and select **Create Key**.
  4. Copy the key and set:

     ```bash
     export MISTRAL_API_KEY="<your-mistral-key>"
     ```

> **Tip:** You can store these in a `.env` file and load them in Python with `python-dotenv` for convenience.


In [8]:
import os, getpass


os.environ["LANGSMITH_API_KEY"] ="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"
os.environ["TAVILY_API_KEY"]  = "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"
os.environ["MISTRAL_API_KEY"] = "C69TRtdJxynci7Bf0RVUENq2vitNr81n"
# LangSmith (for tracing/debugging your agent’s steps)
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API key: ")

# Tavily Search
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")

# Mistral
if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Mistral API key: ")


LangSmith API key: ··········
Tavily API key: ··········


## 4. Build a Search Tool

LangChain’s **`Tool`** abstraction lets your agent call out to external capabilities (search, databases, calculators, etc.) as part of its thought–action cycle. Behind the scenes, each tool is just a Python function or class with a `run` method, plus a human-readable description.


In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults #This class is provided by the `langchain-community` package as a ready-made wrapper over the Tavily Search API.

# Return up to 3 results when invoked
search_tool = TavilySearchResults(max_results=3)

# You can add more tools (calculator, SQL, custom API wrappers…) to this list.
tools = [search_tool]


* **`max_results=3`** tells the tool to fetch at most three search hits for each query.
* Internally, calling `search_tool.run(query_string)` will:

  1. Send your query to Tavily’s API
  2. Receive a JSON payload of results (title, URL, snippet)
  3. Return a formatted list (or string) your agent can parse

  

In [10]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "mistral-instruct-v0.3",        # lighter, more available model
    model_provider="mistralai",
)



In [11]:
model_with_tools = model.bind_tools(tools)


When your agent processes a user question, it may decide: “I need more information, so I’ll call the **search\_tool**.” Under the hood, LangChain:

1. Inserts an action step into its reasoning chain: `Action: search_tool.run`
2. Passes the user’s query into `search_tool.run(query)`
3. Receives and records the results
4. Continues reasoning to produce a final answer that incorporates the search results


In [12]:
# 1) Imports
import os
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain.agents import initialize_agent, Tool, AgentType

# 2) Define a simple calculator function
def calculate_expression(expr: str) -> str:
    """
    Safely evaluate a basic math expression and return the result.
    Supports +, -, *, /, **, parentheses.
    """
    try:
        # Only allow digits, operators, and whitespace
        allowed = "0123456789+-*/() .**"
        if any(c not in allowed for c in expr):
            return "Error: Invalid characters in expression."
        result = eval(expr, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

# 3) Wrap it as a LangChain Tool
calc_tool = Tool(
    name="calculator",
    func=calculate_expression,
    description="Use this to compute math expressions, e.g. '23 * (7 + 5)**2'",
)

# 4) Initialize your Mistral chat model
model = init_chat_model(
    "mistral-large-latest",
    model_provider="mistralai",
    # ensure MISTRAL_API_KEY is set in your env
)

# 5) Build a ReAct-style agent with the calculator tool
agent = initialize_agent(
    tools=[calc_tool],
    llm=model,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# 6) Ask the agent a math question!
response = agent.invoke({"input": "What is 23 * (7 + 5)**2?"})
print(response)


/tmp/ipython-input-12-3116106650.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...


Question: What is 23 * (7 + 5)**2?
Thought: I need to calculate the value of 23 * (7 + 5)**2.
Action:
```
{
  "action": "calculator",
  "action_input": "23 * (7 + 5)**2"
}
```
Observation: 3312
Thought:The observation from the calculator tool is 3312.

Final Answer: 3312

> Finished chain.
{'input': 'What is 23 * (7 + 5)**2?', 'output': '3312'}


1. **Reasoning**: The agent examines the user’s input and decides a calculation is needed.
2. **Action**: It generates an action step like `Action: calculator(expression)`.
3. **Execution**: LangChain calls `calculate_expression(expression)` under the hood.
4. **Result**: The function returns a result string, which the agent incorporates into its next reasoning step and final answer.

In [13]:
from langgraph.prebuilt import create_react_agent

# Create a higher-level executor
agent_executor = create_react_agent(model, tools)

# Simple chat
out = agent_executor.invoke({"messages":[HumanMessage(content="Hi there!")]})
print(out["messages"][-1].content)

# Ask it to use the search tool
out = agent_executor.invoke({"messages":[HumanMessage(content="Find me the latest news on AI agents.")]})
print(out["messages"][-1].content)


Hello! How can I assist you today?
[{'type': 'text', 'text': "Here are some of the latest insights and news on AI agents:\n\n1. **IBM's Perspective on AI Agents in 2025**:\n   - IBM experts, including Maryam Ashoori, Marina Danilevsky, Vyoma Gajjar, and Chris Hay, discuss the realistic expectations for AI agents in 2025. They aim to provide a balanced view beyond the hype, emphasizing that while there is excitement about the potential of AI agents, it's important to have a reasonable conversation about their capabilities and impacts. Industry reports suggest that 2025 could see a rise in the number and effectiveness of AI agents, with a focus on their autonomous capabilities and potential for profitability "}, {'type': 'reference', 'reference_ids': [1]}, {'type': 'text', 'text': '.\n\n2. **How AI Agents Work in 2025**:\n   - An article on Medium by Berto Mill highlights the transition of AI from simple task automation to complex, multi-step operations performed by sophisticated agentic

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="What’s the weather in SF?")]},
    stream_mode="values",
):
    # Each ‘step’ contains the newest message(s)
    step["messages"][-1].pretty_print()


================================ Human Message =================================

What’s the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (9r4bZYX61)
 Call ID: 9r4bZYX61
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "San Francisco weather in July 2025 | Weather25.com", "url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=July", "content": "| 27 Image 54: Mist 15°/13° | 28 Image 55: Fog 18°/12° | 29 Image 56: Overcast 18°/14° | 30 Image 57: Patchy rain possible 17°/14° | 31 Image 58: Partly cloudy 19°/15° |  |  | [...] Friday Jul 25 Image 8: Mist 0 mm 15°/14°Saturday Jul 26 Image 9: Overcast 0 mm 15°/14°Sunday Jul 27 Image 10: Mist 0 mm 15°/13°Monday Jul 28 Image 11: Fog 0 mm 18°/12°Tuesday Jul 29 Image 12: Overcast 0 mm 18°/14°Wednesday Jul 30 

In [18]:
# a) Ask a math question
resp = agent_executor.invoke({"messages": [HumanMessage(content="What is 23 * (7 + 5)**2?")]})
print(resp)

In [20]:
# b) Ask a search question
resp = agent_executor.invoke({"messages": [HumanMessage(content="Find me the latest news on AI agents.")]})
print(resp)

{'messages': [HumanMessage(content='Find me the latest news on AI agents.', additional_kwargs={}, response_metadata={}, id='987406ed-a424-4ba2-bb3e-75d097bdb5fe'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bUSgxKB4j', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "latest news on AI agents"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 116, 'total_tokens': 148, 'completion_tokens': 32}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run--e88cbde8-69a5-4deb-bed9-835e456b8a31-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'latest news on AI agents'}, 'id': 'bUSgxKB4j', 'type': 'tool_call'}], usage_metadata={'input_tokens': 116, 'output_tokens': 32, 'total_tokens': 148}), ToolMessage(content='[{"title": "Daily AI Agent News - July 2025", "url": "https://aiagentstore.ai/ai-agent-news/2025-july", "content": "AI Agent Store Logo\\n\\

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
agent_executor = create_react_agent(
    model,   # Your initialized ChatModel (e.g., Mistral)
    tools    # The list of Tool objects (search_tool, calc_tool, etc.)
)

In [ ]:
from langchain_core.messages import HumanMessage

# Prepare a single user message
messages = [HumanMessage(content="Hi there!")]

# Invoke synchronously
out = agent_executor.invoke({"messages": messages})

# The output dict contains a "messages" list
# with your HumanMessage plus assistant replies
print(out["messages"][-1].content)

Hello! How can I assist you today?


In [ ]:
from langchain_core.messages import HumanMessage

for step in agent_executor.stream({"messages": [HumanMessage(content="What's the weather in SF?")]},
                                 stream_mode="values"):
    # Each step contains the incremental reply
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather in SF?


HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [ ]:
from langchain_core.messages import HumanMessage

# Prepare your input message
streaming_input = {"messages": [HumanMessage(content="What's the weather in SF?")]}

# Iterate over streamed chunks
for step in agent_executor.stream(
    streaming_input,
    stream_mode="values"
):
    # 'step' is a dict with a "messages" list
    # The last message is the assistant's partial reply
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather in SF?


================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (n1sYldF9J)
 Call ID: n1sYldF9J
  Args:
    query: weather in SF


================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco in May 2025 (California)", "url": "https://world-weather.info/forecast/usa/san_francisco/may-2025/", "content": "Weather in San Francisco in May 2025. San Francisco Weather Forecast for May ... Sunday, 11 May. +52°. Day. +63°. Clear sky. Monday, 12 May. +54°. Day. +63°. Few", "score": 0.94401723}, {"title": "Weather in San Francisco in May 2025 - Detailed Forecast", "url": "https://www.easeweather.com/north-america/united-states/california/city-and-county-of-san-francisco/san-francisco/may", "content": "| 66° /48° | 0 in | 5 |  |\n| May 11 | \nSunny\n| 66° /48° | 0 in | 5 |  |\n| May 12 | \nSunny\n| 64° /48° | 0 in | 5 |  |\n| May 13 | \nSunny\n| 68° /50° | 0 in | 6 |  |\n| May 14 | \nSunny\n| 66° /50° | 0 in | 6 |  |\n| May 15 | \nOvercast\n| 64° /50° | 0 in | 5 |  |\n| May 16 | \nCloudy\n| 64° /50° | 0 in | 5 |  |\n| May 17 | \nOverca